# Crear la clase  Target

## Ejercicio 1

Cree el archivo competencia_01.csv, usando el competencia_01_crudo.csv con una variable adicional llamada clase_ternaria, que contenga las categorías **CONTINUA, BAJA+1 y BAJA+2**

Ayudita
Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**.

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction) Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
# %%bash
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb**

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
select 'hola mundo'

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla pasando su ubicación a través de la variable `dataset_path`:

In [4]:
dataset_path = 'C:/Users/pedro/Documents/DMEyF/dmeyf2025/data/'
dataset_file = 'raw/competencia_01_crudo.csv'

In [5]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [6]:
%sql select * from competencia_01_crudo limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,0.0,4,0.00,-20833.47,0.0,2814,6743.38,6,0,9559.95
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,0.0,11,13267.49,-9662.51,0.0,5541,9419.90,8,0,1900.26
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,0.0,4,0.00,-4692.00,0.0,577,2821.32,2,0,6885.51
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,0.0,25,23800.17,-20349.22,0.0,7916,23509.71,8,0,1219.92
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,0.0,4,0.00,-88178.37,0.0,1958,20397.54,8,0,2627.52


In [7]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_01_crudo
group by foto_mes

,foto_mes,cantidad
0,202101,161527
1,202102,162155
2,202103,163109
3,202104,163418
4,202105,163917
5,202106,164313


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide.

In [8]:
%%time
%%sql
create or replace table competencia_01 as
WITH tabla_1 AS (
    SELECT *,
    LEAD(foto_mes, 1, 0) OVER(PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes_1,
    LEAD(foto_mes, 2, 0) OVER(PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes_2
    FROM competencia_01_crudo
)
SELECT
    * EXCLUDE(mes_1, mes_2),
    CASE
        WHEN foto_mes = 202106 THEN NULL
        WHEN mes_1 = 0 THEN 'BAJA+1'
        WHEN foto_mes = 202105 AND mes_2 = 0 THEN NULL 
        WHEN mes_1 <> 0 AND mes_2 = 0 THEN 'BAJA+2'
        ELSE 'CONTINUA'
    END AS clase_ternaria
FROM tabla_1;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 21.7 s
Wall time: 4.14 s


In [9]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249255747,202101,1,0,0,56,85,792.76,48185.72,397.40,...,4,0.0,-19323.21,0.0,1538,26521.15,18,0,1618.74,CONTINUA
1,249255747,202102,1,0,0,56,86,15549.20,52210.75,15131.87,...,-3,0.0,-3811.58,0.0,1566,18188.18,9,0,3237.48,CONTINUA
2,249255747,202103,1,0,0,56,87,1219.16,51994.65,1229.18,...,1,0.0,-3237.48,0.0,1597,7042.92,7,0,3460.35,CONTINUA
3,249255747,202104,1,0,0,56,88,9023.22,58668.96,8795.06,...,2,0.0,-46651.84,0.0,1627,26086.04,15,0,0.00,CONTINUA
4,249255747,202105,1,0,0,56,89,3307.06,59767.23,1975.12,...,5,0.0,-24860.82,0.0,1658,28443.60,9,0,7683.15,None


## Ejercicio 1.1
* ¿Cuál es la nominalidad de cada clase?


In [10]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,616,824,160087
1,202102,830,1027,160298
2,202103,1034,953,161122
3,202104,954,1131,161333
4,202105,1134,0,0
5,202106,0,0,0


* ¿Cuál es la proporción del target?

## Guardar tabla en .csv

Para guardar a un `.csv` simplemente debe ejecutar la siguiente sentencia

In [12]:
%%sql
COPY competencia_01 TO '{{dataset_path}}/processed/competencia_01.csv' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
